## Preambule

In [54]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json
from tqdm import tqdm
import plotly

# Sobol analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [55]:
standard_settings = dict(Risk_of_exceedance='50%',
                         Negative_emissions='Medium',
                         Non_CO2_mitigation_potential='Medium')
params = dict(start_year_analysis=2021)
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']
rulecolors = ['steelblue', 'goldenrod', 'forestgreen', 'sienna', 'mediumvioletred']
costoptimal_colors = ['orangered', 'red', 'darkred']

## Paths

In [56]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data"
path_data_ext = Path("X:/user/dekkerm/Data")
path_figs = path_main / "Figures" / "ECEMF_paper"

## Read data files

In [57]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Uncertainty decomposition of budgets

In [58]:
problem = {
    'num_vars': 4,
    'names': ['Temperature', 'Convergence_year', 'Scenario', 'EffortSharing'],
    'bounds': [[1.5, 3.0],
               [np.min(xr_total.Convergence_year), np.max(xr_total.Convergence_year)],
               [0, 1],
               [0, 6]]
}

In [59]:
param_values = saltelli.sample(problem, 64)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_9708\357594605.py:1: DeprecationWarning:

`salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`



In [60]:
def refine_sample(pars):
    v1 = pars.copy()[:, 0]
    v1s = v1.astype(str)
    v1s[v1 < 1.6] = '1.5 deg'
    v1s[(v1 < 1.85) & (v1 >= 1.6)] = '1.7 deg'
    v1s[(v1 < 2.25) & (v1 >= 1.85)] = '2.0 deg'
    v1s[(v1 < 2.75) & (v1 >= 2.25)] = '2.5 deg'
    v1s[(v1 >= 2.75)] = '3.0 deg'
    
    v2 = pars.copy()[:, 1]
    v2s = (np.round(v2 / 5) * 5).astype(int)
    
    v3 = pars.copy()[:, 2]
    v3s = v3.astype(str)
    v3s[v3 < 0.2] = 'SSP1'
    v3s[(v3 >= 0.2) & (v3 < 0.4)] = 'SSP2'
    v3s[(v3 >= 0.4) & (v3 < 0.6)] = 'SSP3'
    v3s[(v3 >= 0.6) & (v3 < 0.8)] = 'SSP4'
    v3s[(v3 >= 0.8) & (v3 < 1.0)] = 'SSP5'
    
    v4 = pars.copy()[:, 3]
    v4s = v4.astype(str)
    v4s[v4 < 1] = 'GF'
    v4s[(v4 >= 1) & (v4 < 2)] = 'PC'
    v4s[(v4 >= 2) & (v4 < 3)] = 'PCC'
    v4s[(v4 >= 3) & (v4 < 4)] = 'AP'
    v4s[(v4 >= 4) & (v4 < 5)] = 'GDR'
    v4s[(v4 >= 5) & (v4 < 6)] = 'ECPC'
    return np.array([v1s, v2s, v3s, v4s]).T

In [61]:
def func(pars, reg, xrt):
    xrt2 = xrt.sel(Region=reg)
    vec = np.zeros(len(pars))
    for i in range(len(pars)):
        vec[i] = float(xrt2.sel(Temperature=pars[i, 0],
                               Convergence_year=np.array(pars[i, 1]).astype(int),
                               Scenario=pars[i, 2])[pars[i, 3]])
    return vec

In [62]:
xrt = xr_total.sel(**standard_settings, Time=np.arange(params['start_year_analysis'], 2101)).sum(dim='Time')
#xrt = xr_total.sel(**standard_settings, Time=np.arange(params['start_year_analysis'], 2101)).sum(dim='Time')

In [63]:
Sis = np.zeros(shape=(len(all_countries_iso), 4))
for reg_i, reg in tqdm(enumerate(all_countries_iso)):
    Y = func(refine_sample(param_values), reg, xrt)
    Si = sobol.analyze(problem, Y)
    Sis[reg_i, :] = Si['ST']

197it [02:44,  1.20it/s]


In [64]:
Si_norm = (Sis.T / Sis.sum(axis=1)).T

## Plot

In [65]:
mins = [np.min(Si_norm[:, 0]),
        np.min(Si_norm[:, 3]),
        np.min(Si_norm[:, 1]+Si_norm[:, 2])]
maxs = [np.max(Si_norm[:, 0]),
        np.max(Si_norm[:, 3]),
        np.max(Si_norm[:, 1]+Si_norm[:, 2])]
mins = (np.array(mins)*100).round(0).astype(int).astype(str)
maxs = (np.array(maxs)*100).round(0).astype(int).astype(str)

In [66]:
def clean_data(data_in):
    """
    Cleans data in a format which can be conveniently
    used for drawing traces. Takes a dictionary as the
    input, and returns a list in the following format:

    input = {'key': ['a b c']}
    output = [key, [a, b, c]]
    """
    key = list(data_in.keys())[0]
    data_out = [key]
    for i in data_in[key]:
        data_out.append(list(map(float, i.split(' '))))

    return data_out

conts2 = []
for i in np.arange(35, 51, 5):
    conts2.append({str(i): [str(i) +' '+str(i)+' '+str(100-2*i),
                  str(i) +' '+str(100-2*i)+' '+str(i),
                  str(100-2*i) +' '+str(i)+' '+str(i)]})
for x in np.arange(55, 101, 5):
    conts2.append({str(i): [str(x)+' '+'0'+' '+str(100-x),
                               str(x-5)+' '+'0'+' '+str(100-(x-5)),
                               str(x-5)+' '+str(100-(x-5))+' '+'0',
                               str(x)+' '+str(100-x)+' '+'0',
                               str(x)+' '+'0'+' '+str(100-x)
                                ]})
    conts2.append({str(i): ['0'+' '+str(100-x)+' '+str(x),
                               '0'+' '+str(100-(x-5))+' '+str(x-5),
                               str(100-(x-5))+' '+'0'+' '+str(x-5),
                               str(100-x)+' '+'0'+' '+str(x),
                               '0'+' '+str(100-x)+' '+str(x)
                                ]})
    conts2.append({str(i): [str(100-x)+' '+str(x)+' '+'0',
                               str(100-(x-5))+' '+str(x-5)+' '+'0',
                               '0'+' '+str(x-5)+' '+str(100-(x-5)),
                               '0'+' '+str(x)+' '+str(100-x),
                               str(100-x)+' '+str(x)+' '+'0'
                                ]})

bar = px.colors.sequential.Greys
colors = []
for i in range(len(bar)-1):
    colors = colors+n_colors(bar[i], bar[i+1], int(200/len(bar)), colortype='rgb')
colors = colors+n_colors(bar[i+1], bar[i+1], int(200/len(bar)), colortype='rgb')

conttraces = []
n = 0
for raw_data in conts2:
    data = clean_data(raw_data)

    a = [inner_data[0] for inner_data in data[1:]]
    a.append(data[1][0]) # Closing the loop

    b = [inner_data[1] for inner_data in data[1:]]
    b.append(data[1][1]) # Closing the loop

    c = [inner_data[2] for inner_data in data[1:]]
    c.append(data[1][2]) # Closing the loop

    conttraces.append(go.Scatterternary(
        text = data[0],
        a=a, b=b, c=c, mode='lines',
        line=dict(color='black', width=0),
        fill='toself',showlegend=False, hoverinfo='skip', opacity=0.2, 
        fillcolor = colors[int(200/len(conts2)*n)]#colors_iterator.__next__()
    ))
    n+=1
conttraces = conttraces[::-1]

In [67]:
args = dict(xref="paper", yref="paper", showarrow=False)
annotations = [
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.005, x=0.05, text="<b>(a) Drivers of fair shares</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=1.005, x=0.92, text="<b>(b) Temperature goal ("+mins[0]+"%-"+maxs[0]+"%)</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.67, x=0.95, text="<b>(c) Effort sharing method ("+mins[1]+"%-"+maxs[1]+"%)</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18), y=0.315, x=0.97, text="<b>(d) Scenario and parameters ("+mins[2]+"%-"+maxs[2]+"%)</b>"),
    dict(text="Temperature", x=0.27, y=1.03, **args, font=dict(color='darkgray', size=25)),
    dict(text="main driver", x=0.29, y=1.00, **args, font=dict(color='darkgray', size=16)),
    dict(text="Effort sharing", x=-0.11, y=0.03, **args, font=dict(color='darkgray', size=25)),
    dict(text="method", x=-0.08, y=0.0, **args, font=dict(color='darkgray', size=25)),
    dict(text="main driver", x=-0.08, y=-0.02, **args, font=dict(color='darkgray', size=16)),
    dict(text="Scenario and", x=0.78, y=0.0, **args, font=dict(color='darkgray', size=25)),
    dict(text="parameters", x=0.77, y=-0.03, **args, font=dict(color='darkgray', size=25)),
    dict(text="main driver", x=0.75, y=-0.04, **args, font=dict(color='darkgray', size=16))
]

In [68]:
df_regions = pd.read_csv(path_data_ext / "AllCountries.csv")

In [69]:
continents = []
bubblecolors = []
for cty_i, cty in enumerate(all_countries_iso):
    cont = np.array(df_regions.region[df_regions['alpha-3'] == cty])[0]
    continents.append(cont)
    if cont == 'Africa':
        bubblecolors.append('forestgreen')
    elif cont == 'Europe':
        bubblecolors.append('steelblue')
    elif cont == 'Asia':
        bubblecolors.append('orange')
    elif cont == 'Americas':
        bubblecolors.append('plum')
    elif cont == 'Oceania':
        bubblecolors.append('goldenrod')
    else:
        bubblecolors.append('silver')
continents = np.array(continents)
bubblecolors = np.array(bubblecolors)

In [70]:
pops = np.array(xr_total.Population.sel(Region=all_countries_iso, Time=2020, Scenario='SSP2'))
pops[np.isnan(pops)] = 1
pops_n = pops/np.nanmax(pops)

In [71]:
fig = make_subplots(rows=3, cols=3, specs=[[{"type": "scatterternary", "colspan": 2, "rowspan": 3}, {}, {'type': 'choropleth'}],
                                           [{}, {}, {'type': 'choropleth'}],
                                           [{}, {}, {'type': 'choropleth'}]],
                    horizontal_spacing = 0, vertical_spacing=0.0)

# Panel a
fig.add_trace(go.Scatterternary(a=[0.5, 0.5, 0, 0.5], b=[0, 0.5, 0.5, 0], c=[0.5, 0, 0.5, 0.5],
                                                mode='lines',
                                                showlegend=False,
                                                hoverinfo='skip',
                                                marker={'size': 15,
                                                        'color': 'black',
                                                        'opacity': 1,#0.9-i*0.03,
                                                        'line' :dict(width=0., color='silver')},
                                                line={'width': 1, #np.mean(s[i:i+2])*0.5,
                                                        'color': 'black',
                                                        'dash': 'dot'},
                                                textfont=dict(size=5,
                                                        color='black')), 1, 1)

for cont_i, cont in enumerate(np.unique(continents)):
    fig.add_trace(go.Scatterternary(a=Si_norm[:, 0], b=Si_norm[continents == cont, 3], c=Si_norm[continents == cont, 1]+Si_norm[continents == cont, 2],
                                                    mode='markers+text',
                                                    showlegend=False,
                                                    text=all_countries_iso[continents == cont],
                                                    marker={'size': (pops_n[continents == cont])**0.4*60,
                                                            'color': bubblecolors[continents == cont],
                                                            'opacity': 0.7,#0.9-i*0.03,
                                                            'line' :dict(width=0., color='silver')},
                                                    line={'width': 1, #np.mean(s[i:i+2])*0.5,
                                                            'color': 'black',
                                                            'dash': 'dot'},
                                                    textfont=dict(size=(pops_n[continents == cont])**0.4*25+1,
                                                            color='black')), 1, 1)
for n in range(len(conttraces)):
        fig.add_trace(conttraces[n], 1, 1)

fig.update_layout({
'plot_bgcolor':'rgb(243, 243, 243)',
'ternary':
{'sum':1,
'bgcolor':'rgba(0,0,0,0)',#'whitesmoke',
    # "paper_bgcolor":'rgba(0,0,0,0)',
    # "plot_bgcolor":'rgba(0,0,0,0)',
'aaxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},#Climate target<br>
        'linewidth':0, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'], 'tickfont':{'size':15}},
'baxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},# &nbsp; &nbsp; &nbsp;#Model &nbsp; &nbsp;
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':60, 'tickfont':{'size':15}},
'caxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},#Other<br>scenario assumptions
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':-60, 'tickfont':{'size':15}}}})


# Panel b-d
for frac_i, frac in enumerate(['Temperature', 'EffortSharing', 'Parameters']):
    if frac_i == 0:
        t = Si_norm[:, 0]
    if frac_i == 1:
        t = Si_norm[:, 3]
    if frac_i == 2:
        t = Si_norm[:, 1]+Si_norm[:, 2]
    fig.add_trace(
        go.Choropleth(
        locations=all_countries_iso,
        z = t,
        locationmode = 'ISO-3',
        colorscale = 'Spectral_r',#'RdYlGn_r',
        zmax = 1,
        zmin = 0,
        text = [str(r)+": "+str(np.round(int(100*t[r_i])))+"%" for r_i, r in enumerate(all_countries_iso)],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='white', 
        marker_line_width=0.2,
        colorbar=dict(len=1, x=1, tickvals = np.arange(0, 1.01, 0.1), ticktext=[str(int(100*i))+'%' for i in np.arange(0, 1.01, 0.1)],
                      tickfont={'size':17}, title={"text":"<b>Percentage<br>variance<br>explained", 'font': {'size':17}}),
        showscale=[True, False, False][frac_i]
    ), [1, 2, 3][frac_i], [3, 3, 3][frac_i])

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)

fig.update_geos(visible=True, showcountries=False)

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    template='ggplot2',
    height=900,
    width=1600,
)
fig.update_xaxes(visible=False)

for ann in annotations:
    fig.add_annotation(**ann)
fig.update_layout(margin={"r":0,"t":30,"l":150,"b":50})
fig.show()

ValueError: cannot convert float NaN to integer

In [ ]:
fig.write_image(path_figs / "Figure_4.png", scale=3)